# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [49]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [50]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [52]:
# handle missing data
spaceship_clean = spaceship.dropna()

In [53]:
#your code here
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Feature Selection
features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
target = 'Transported'

# Feature Scaling
X = spaceship[features]
y = spaceship[target]

# Impute missing values
imputer = SimpleImputer(strategy='mean')  # Puedes cambiar 'mean' por 'median' o 'most_frequent'
X_imputed = imputer.fit_transform(X)

# Feasture Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [54]:
# Perform Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(X_train.shape)  

(6954, 6)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [56]:
#your code here
from sklearn.ensemble import GradientBoostingClassifier
# Gradient Boosting
gb_clf = GradientBoostingClassifier(
    n_estimators=100,             
    learning_rate=0.1,            
    max_depth=3,                  
    random_state=42
)

# Train Gradient Boosting
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
acc_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {acc_gb:.2f}")

Gradient Boosting Accuracy: 0.78


- Evaluate your model

In [58]:
#your code here
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Gradient Boosting
gb_clf = GradientBoostingClassifier(
    n_estimators=100,             
    learning_rate=0.1,            
    max_depth=3,                  
    random_state=42
)

# Train Gradient Boosting
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
acc_gb = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Accuracy: {acc_gb:.2f}")

Gradient Boosting Accuracy: 0.78


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [62]:
# Define the hyperparameter search space
param_grid = {
    'n_estimators': [50, 100, 200],         # Number of trees in the ensemble
    'learning_rate': [0.01, 0.1, 0.2],     # Learning rate shrinkage
    'max_depth': [3, 4, 5],                # Maximum depth of each tree
    'subsample': [0.8, 1.0],               # Fraction of samples to be used for fitting each base learner
}

- Run Grid Search

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Set up the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(random_state=42)

# Set up the Grid Search
grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Train using Grid Search
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]},
             verbose=2)

- Evaluate your model

In [66]:
from sklearn.metrics import accuracy_score

# Print the best classifier results
print(f"Best parameters: {grid_search.best_params_}")
best_gb_clf = grid_search.best_estimator_

# Evaluate the tuned model
y_pred_gb = best_gb_clf.predict(X_test)
acc_gb = accuracy_score(y_test, y_pred_gb)
print(f"Tuned Gradient Boosting Accuracy: {acc_gb:.2f}")

Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Tuned Gradient Boosting Accuracy: 0.77
